<a href="https://colab.research.google.com/github/kentokura/ox_2x2_retrograde_analysis/blob/main/ox2x2/makeAllState.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ノードの３状態:
  - 未発見
    unsolved, solvedのいずれにもnext_nodeが存在しない
  - 未訪問
    unsolvedに存在する
  - 既訪問
    solvedに存在する

処理:
- 初期状態"____"を未訪問にする.
- 未訪問キューが空になるまで以下を行う # BFS開始
  1. 未訪問キューから先頭のノードをcurrent_nodeとしてpopする.
  1. 盤面の勝敗判定を行う。
    1. もし、勝敗がついている盤面なら、
      1. RESULTに勝敗を記録
    1. 勝敗がついていない盤面なら
      1. current_nodeを探索し、次のノード(next_nodes)全てをnext_nodesとして列挙する.
      1. next_nodesの要素next_nodeそれぞれに対して、以下を行う.
        1. もし、next_nodeが未発見ならば、
          1. そのノードを未訪問にする.
        1. そうではなく、発見済みならば
          1. 特になし.
        1. next_node.previous_nodeにcurrent_nodeを追加する.
        1. current_node.next_node.にnext_nodeを追加する.
  1. ノード（current_node）を既訪問にする.

1. solvedをcsvで書き出し.

# インプット

In [1]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# csvの読み込むためのモジュール
import pandas as pd
from pandas import DataFrame
import numpy as np
from tabulate import tabulate # pandasのdfをきれいに出力するためのモジュール

# 処理

In [18]:
### BFSでゲーム木を作成するプログラム ### 

###

### 関数群ここから ###

# stateを入力すると、その盤面の勝敗を返す関数
def resultcheck(state: str) -> str:
  # return 一覧
    # "o_win"
    # "x_win"
    # "draw"
    # ""
  result = ""
  if len(state) == 4:
    state = list(state)
    if (state[0] != "_" and (state[0] == state[1] or state[0] == state[2] or state[0] == state[3])) or (state[1] != "_" and (state[1] == state[2] or state[1] == state[3] )) or (state[2] != "_" and state[2] == state[3]):
      result = "{}_win".format(state[0])
    elif state in ["oooo", "xxxx"]:
      result = "draw"

  return result

# stateを入力すると、正規化したstateを返す関数
def normalization(state: str) -> str: 
    normalization_state = state
    if state in ["o___", "_o__", "__o_", "___o"]:
      normalization_state = "o___"
    elif state ==  ["ox__", "o_x_", "_o_x", "xo__", "_x_o", "__xo", "x_o_", "__ox"]:
      normalization_state ="ox__"
    elif state ==  ["o__x", "_ox_", "x_o_", "_x_o"]:
      normalization_state = "o__x"
    elif state ==  ["x_oo", "xo_o", "oxo_", "_xoo", "oox_", "_oxo", "oo_x", "o_ox"]:
      normalization_state = "oxo_"
    elif state ==  ["xoo_", "ox_o", "o_xo", "_oox"]:
      normalization_state = "ox_o"
    elif state ==  ["oxox", "xxoo", "xoxo", "ooxx"]:
      normalization_state = "oxox"
    elif state ==  ["oxxo", "xoox"]:
      normalization_state = "oxxo"

    return normalization_state

# state を入力すると、次のstateのリストを出力する関数

def nextStates(state: str) -> list: 
  next_states = []
  # 次の手番の決定
  player = "_"
  if state.count('o') <= state.count('x'):
    player = "o"
  else:
    player = "x"
  # next_nodeの作成
  for i, piece in enumerate(state):
    if piece == "_":
      next_state = list(state)
      next_state[i] = player
      next_states.append(''.join(next_state))
      # print(next_states)
  # return next_states
  # next_statesに登録された全てのstateを正規化
  normalization_next_states = []
  for next_state in next_states:
    normalization_next_state = normalization(next_state)
    normalization_next_states.append(''.join(normalization_next_state))
  # 重複削除
  normalization_next_states = list(set(normalization_next_states))
  return normalization_next_states

### 関数群ここまで

### 設定ここから ###
printFlag = False

### 設定ここまで ###

###mainここから

# unsolvedDf, solvedDfの初期化
if printFlag:
  print("===")
  print("プログラム開始")
  print("===")
  print()
  print("データを初期化します")
cols = ["PREVIOUS_STATES", "STATE", "NEXT_STATES", "RESULT"] #[前の状態list， 状態, 次の状態list]
df = pd.DataFrame(index=[], columns=cols)
df.set_index("STATE")
unsolvedDf = df
solvedDf = df
if printFlag:
  print("データを初期化しました")
  print()

# 初期状態"____"をunsolvedに追加する。unsolvedに積まれているノードは未訪問.
if printFlag:
  print("===")
  print("BFSの準備")
  print("===")
  print()
  print("初期状態をセットします")
init_state = "____"
previous_state = ""
unsolvedDf = unsolvedDf.append(pd.Series([[previous_state], init_state, "unsolved", ""], index=df.columns, name=init_state))
if printFlag:
  print("初期状態をセットしました") # 確認
  print("確認[UNSOLVED_DF]:") # 確認
  print(unsolvedDf) # 確認
  print() # 確認

# unsolvedが空になるまで以下を行う. BFS開始
if printFlag:
  print("===")
  print("BFSを開始します")
  print("===")
  print()
for _ in range(1000): # while len(unsolvedDf) > 0: # 開発のためにfor文にしている。
  # unsolvedDfから先頭のノードをpopする。
  if len(unsolvedDf) <= 0:
    break;
  current_node = unsolvedDf.iloc[0]  # 先頭のノード(current_node)を抽出。
  unsolvedDf.drop(unsolvedDf.index[0], inplace=True)  # 抽出したノードをunsolvedから削除。
  # 勝敗の確認
  result = resultcheck(current_node.STATE)
  # 勝敗確定盤面なら
  if result != "":
    current_node.RESULT = result
    current_node.NEXT_STATES = []
  else:  # 勝敗確定盤面でないなら
    # 先頭のノード(current_node)から次のノード(next_nodes)を探索する。
    next_states = nextStates(current_node.STATE) # 次のノードの探索結果
    current_node.NEXT_STATES = next_states # current_nodeのNEXT_STATESに探索結果を反映
    # 探索した全ての状態について、以下を行う。
    if printFlag:
      print("unsolvedDfからpopされたノード'{}'の探索を行います".format(current_node.STATE))
    if len(next_states) <= 0:
      if printFlag:
        print("    探索結果: このノードは末端です")
    for next_state in next_states:
      # もし、next_nodeが未発見ならば # unsolved, solvedのいずれにもnext_nodeが存在しない
      if (next_state not in unsolvedDf.STATE.values) and (next_state not in solvedDf.STATE.values):
        if next_state == current_node.STATE: # 次のノードが自身と同一
          if printFlag:
            print("探索結果: 自身のノード'{}'と同一です".format(next_state))
          continue;
        else:
          if printFlag:
            print("    探索結果: 未発見のノード'{}'です".format(next_state))
          # T)そのノードを未訪問にする。 # unsolvedに追加
          previous_state = [current_node.STATE]
          next_node = pd.Series([previous_state, next_state, "unsolved", ""], index=df.columns, name=next_state) # next_nodeの作成
          unsolvedDf = unsolvedDf.append(next_node)
      else:  # F)そうではなく、発見済みならば
        if printFlag:
          print("    探索結果: 発見済みのノード'{}'です".format(next_state))
        #これを既に登録されていたノードのprevious_stateに追加する。
        previous_state = [current_node.STATE] 
        if next_state in unsolvedDf.STATE.values: # unsolvedDfに存在
          if printFlag:
            print("        これはunsolvedに存在しています")
          # unsolvedDf[unsolvedDf.STATE.values == next_state])にprevious_stateを追加する
          tmp = unsolvedDf.loc[next_state, "PREVIOUS_STATES"]
          tmp.append(previous_state[0])       
          unsolvedDf.loc[next_state, "PREVIOUS_STATES"] = tmp
        elif next_state in solvedDf.STATE.values:# solveDfに存在
          if printFlag:
            print("        これはsolvedに存在しています")
          # solvedDf[solvedDf.STATE.values == next_state])にprevious_stateを追加する
          tmp = solvedDf.loc[next_state, "PREVIOUS_STATES"]
          tmp.append(previous_state[0])       
          solvedDf.loc[next_state, "PREVIOUS_STATES"] = tmp
        else: # 何らかの理由で漏れた状態
          print("        エラー")
  # 現在のノード（current_node）をsolvedDfに追加する。solvedDfのノードは既訪問。 
  solvedDf = solvedDf.append(current_node)

if printFlag:
  print()
  print("BFSが終了しました")
  print()

# 結果確認
print("===")
print("結果確認")
print("===")
print()
print("確認[unsolvedDf]:")
print()
print(tabulate(unsolvedDf, unsolvedDf.columns,tablefmt='github', showindex=True))
print()
print("確認[solvedDf]:")
print()
print(tabulate(solvedDf, solvedDf.columns,tablefmt='github', showindex=True))
print()


### mainここまで

===
結果確認
===

確認[unsolvedDf]:

| PREVIOUS_STATES   | STATE   | NEXT_STATES   | RESULT   |
|-------------------|---------|---------------|----------|

確認[solvedDf]:

|      | PREVIOUS_STATES   | STATE   | NEXT_STATES              | RESULT   |
|------|-------------------|---------|--------------------------|----------|
| ____ | ['']              | ____    | ['o___']                 |          |
| o___ | ['____']          | o___    | ['o__x', 'o_x_', 'ox__'] |          |
| o__x | ['o___']          | o__x    | ['oo_x', 'o_ox']         |          |
| o_x_ | ['o___']          | o_x_    | ['o_xo', 'oox_']         |          |
| ox__ | ['o___']          | ox__    | ['oxo_', 'ox_o']         |          |
| oo_x | ['o__x']          | oo_x    | []                       | o_win    |
| o_ox | ['o__x']          | o_ox    | []                       | o_win    |
| o_xo | ['o_x_']          | o_xo    | []                       | o_win    |
| oox_ | ['o_x_']          | oox_    | []                       |

### 出力結果

確認[solvedDf]:

|      | PREVIOUS_STATES   | STATE   | NEXT_STATES              | RESULT   |
|------|-------------------|---------|--------------------------|----------|
| ____ | ['']              | ____    | ['o___']                 |          |
| o___ | ['____']          | o___    | ['o__x', 'o_x_', 'ox__'] |          |
| o__x | ['o___']          | o__x    | ['oo_x', 'o_ox']         |          |
| o_x_ | ['o___']          | o_x_    | ['o_xo', 'oox_']         |          |
| ox__ | ['o___']          | ox__    | ['oxo_', 'ox_o']         |          |
| oo_x | ['o__x']          | oo_x    | []                       | o_win    |
| o_ox | ['o__x']          | o_ox    | []                       | o_win    |
| o_xo | ['o_x_']          | o_xo    | []                       | o_win    |
| oox_ | ['o_x_']          | oox_    | []                       | o_win    |
| oxo_ | ['ox__']          | oxo_    | []                       | o_win    |
| ox_o | ['ox__']          | ox_o    | []                       | o_win    |

# 出力

In [19]:
# solvedDfをox_outputという名前で書き出し
solvedDf.to_csv('/content/drive/My Drive/ox/workspace/ox_output.csv')

In [20]:
# ox_outputの確認

solvedDf = pd.read_csv(
    "/content/drive/My Drive/ox/workspace/ox_output.csv",
    index_col=0, # 最初の１行はデータ名。
    encoding="cp932" # windowsの追加文字に対応。おまじないだと思えば良い。
    )
print(solvedDf)

     PREVIOUS_STATES STATE               NEXT_STATES RESULT
____            ['']  ____                  ['o___']    NaN
o___        ['____']  o___  ['o__x', 'o_x_', 'ox__']    NaN
o__x        ['o___']  o__x          ['oo_x', 'o_ox']    NaN
o_x_        ['o___']  o_x_          ['o_xo', 'oox_']    NaN
ox__        ['o___']  ox__          ['oxo_', 'ox_o']    NaN
oo_x        ['o__x']  oo_x                        []  o_win
o_ox        ['o__x']  o_ox                        []  o_win
o_xo        ['o_x_']  o_xo                        []  o_win
oox_        ['o_x_']  oox_                        []  o_win
oxo_        ['ox__']  oxo_                        []  o_win
ox_o        ['ox__']  ox_o                        []  o_win
